<a href="https://colab.research.google.com/github/Yili-Zhang/hello-world/blob/master/Drone_Roberta_Classification_V0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
"""
Install this
"""

!pip install pytorch-transformers

     |████████████████████████████████| 184kB 19.4MB/s 
     |████████████████████████████████| 1.1MB 28.9MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
     |████████████████████████████████| 133kB 55.8MB/s 
     |████████████████████████████████| 6.9MB 51.6MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=dc29c7dd645985d250b4f52df546386eb5a63cdab51343577904f1074d374d3d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.23 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [14]:
"""
Import Libraries
"""
import pandas as pd
import numpy as np

## Torch Modules
import torch
from pytorch_transformers import RobertaTokenizer
from pytorch_transformers import RobertaForSequenceClassification, RobertaConfig

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
"""
Make Sure Cuda is Available
"""

print(torch.cuda.is_available())

True


In [16]:
"""
Loading Configurations
"""
label_to_ix = {'goForward': 0, 'goToRoom': 1}
config = RobertaConfig.from_pretrained('roberta-base')
config.num_labels = len(list(label_to_ix.values()))

"""
Loading Pretrained tokenizer and instantiating the model from settings in config
"""
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification(config)

100%|██████████| 456318/456318 [00:00<00:00, 18624037.02B/s]


In [17]:
"""
Model Path
"""
model_path = 'drive/My Drive/AutoDrone-NLP-Classification-Engine-V0/roberta_state_dict_ef8ac38a-77c6-4139-bed1-70dcab601390.pth'

'''
Loading State dictionaries
'''
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.cuda()
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [18]:
"""
Important Feature Engineering
"""

def prepare_features(seq_1, max_seq_length = 300, 
             zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [19]:
"""
Get Reply
"""

def get_reply(msg):
  model.eval()
  input_msg, _ = prepare_features(msg)
  if torch.cuda.is_available():
    #Send Input Message to CUDA
    #Line - 1
    sent = input_msg.cuda()
  #Use the model to generate probability vector  
  #Line - 2
  output = model.forward(sent)[0]
  #print(output)

  #Use torch.max function to return the prediction index with highest probability. Check how to use torch.max?
  #Line - 3
  _, predicted = torch.max(output, 1)

  #Match prediction index to actual prediction label. Use the label_to_ix = {} dictionary we created earlier.
  #Line - 4
  for key, value in label_to_ix.items():
    if predicted == value:
        prediction = key

  return prediction

In [22]:
"""
Test your function on sentences
"""
get_reply(input("Enter your sentence: "))

Enter your sentence: be in livingroom right now


'goToRoom'